In [1]:
import os
import os.path as op
import mne
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

In [2]:
subjects = ['TD001', 'TD005', 'TD009']
nights = ['N1', 'N2', 'N3', 'N4']

signal = ['epo', 'raw']
reference = ['lobes', 'areas', 'rois']
method = ['source', 'geodesic', 'spherical']

In [49]:
results_dir = '/home/ruggero.basanisi/results/closedloop/topographies'

sbj = subjects[1]
n = nights[2]

sig = signal[1]
ref = reference[0]
met = method[0]

topo_fname = op.join(results_dir, sbj, n, f'{sig}_topo_{ref}_{met}.nc')
topo_data = xr.load_dataarray(topo_fname)

In [50]:
topo_data

<xarray.DataArray (rois: 10, channels: 256, times: 5001)>
array([[[-0.01845467, -0.01845431, -0.01845375, ..., -0.01843669,
         -0.01843601, -0.01843537],
        [-0.01105089, -0.01105012, -0.01104983, ..., -0.01102739,
         -0.01102667, -0.0110262 ],
        [-0.02547531, -0.02547472, -0.02547433, ..., -0.02547351,
         -0.02547309, -0.02547261],
        ...,
        [-0.02471858, -0.02471866, -0.02471811, ..., -0.02471566,
         -0.02471533, -0.02471407],
        [-0.0256946 , -0.02569453, -0.025694  , ..., -0.02569259,
         -0.02569239, -0.02569116],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.02211881, -0.02211887, -0.0221188 , ..., -0.02212097,
         -0.02212102, -0.02212109],
        [-0.01633854, -0.01633859, -0.01633852, ..., -0.01632646,
         -0.01632649, -0.01632657],
        [-0.02569381, -0.02569388, -0.02569382, ..., -0.02570014,
         -0.02570023, -0.02570026],
...
        [-0.02739488, -0.02739497, -0.02739472, ..., -0.0273987 ,
         -0.0273988 , -0.02739891],
        [-0.02834361, -0.02834368, -0.02834348, ..., -0.02834955,
         -0.02834967, -0.02834974],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.02199842, -0.02199854, -0.02199872, ..., -0.02200819,
         -0.02200822, -0.02200815],
        [-0.01618796, -0.01618814, -0.01618828, ..., -0.01619259,
         -0.01619263, -0.01619255],
        [-0.02585035, -0.02585053, -0.02585069, ..., -0.02585989,
         -0.02585988, -0.02585988],
        ...,
        [-0.02751726, -0.02751727, -0.02751743, ..., -0.02752071,
         -0.02752097, -0.0275208 ],
        [-0.02858566, -0.02858568, -0.02858586, ..., -0.02859188,
         -0.02859219, -0.02859205],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
  * rois      (rois) object 'Cingulate-lh' 'Frontal-lh' ... 'Temporal-rh'
  * channels  (channels) object 'Z1L' 'Z2L' 'Z3L' 'Z4L' ... 'Z19Z' 'Z20Z' 'Z12Z'
  * times     (times) float64 -8.0 -7.998 -7.996 -7.994 ... 1.996 1.998 2.0
Attributes:
    sfreq:        500.0
    highpass:     0.0
    lowpass:      250.0
    ch_names:     ['Z1L', 'Z2L', 'Z3L', 'Z4L', 'Z5L', 'Z6L', 'Z7L', 'Z8L', 'Z...
    ch_x_pos:     [-1.47479996e-02 -1.44029995e-02 -1.45370001e-02 -1.4395000...
    ch_y_pos:     [ 0.111706  0.110679  0.10662   0.097543  0.085672  0.06962...
    ch_z_pos:     [ 0.051724    0.072162    0.093639    0.11059     0.125232 ...
    coord_frame:  head
    nasion:       [0.0000000e+00 1.4553558e+21 0.0000000e+00]
    lpa:          [-8.752002e-10  0.000000e+00  0.000000e+00]
    rpa:          [-3.7993926e-31  0.0000000e+00  0.0000000e+00]

In [78]:
roi = 'Occipital-rh'
data = topo_data.sel({'rois': roi})

info = mne.create_info(ch_names=topo_data.ch_names, 
                       sfreq=topo_data.sfreq, 
                       ch_types='eeg')

ch_pos = {k: [x, y, z] 
          for k, x, y, z in zip(topo_data.ch_names, 
                                topo_data.ch_x_pos, 
                                topo_data.ch_y_pos, 
                                topo_data.ch_z_pos)}

mont = mne.channels.make_dig_montage(ch_pos=ch_pos, nasion=topo_data.nasion, 
                                     lpa=topo_data.lpa, rpa=topo_data.rpa, 
                                     hsp=None, hpi=None, coord_frame='head')

tmin = float(topo_data.times[0])


evk = mne.EvokedArray(data, info, tmin=tmin, kind='average', baseline=None)
evk.set_montage(mont)
if sig == 'raw':
    evk = evk.filter(.2, 4.)


Setting up band-pass filter from 0.2 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 8251 samples (16.502 s)



/tmp/ipykernel_2659407/2248377904.py:24: RuntimeWarning: filter_length (8251) is longer than the signal (5001), distortion is likely. Reduce filter length or filter a longer signal.
  evk = evk.filter(.2, 4.)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 256 out of 256 | elapsed:    0.3s finished


In [79]:
# plt.rcParams["figure.figsize"] = (50,50)
# fig = evk.plot_topomap(0)
# fig.set_size_inches(5,5)
# fig.show()
fig, axs = plt.subplots(1, 1, figsize=(4, 4))
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
evk.plot_topomap([0], sphere='auto', axes=[axs], colorbar=False)
plt.title(roi)

Text(0.5, 1.0, 'Occipital-rh')

: 